In [4]:
import re
from stemming.porter2 import stem
from functools import reduce
import math
import gensim
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import collections
from collections import Counter
import string
import scipy
from scipy import sparse
import sklearn
from sklearn import svm
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd


In [ ]:
#doc_dic = dict(system_results[system_results["system_number"] == 1].query_number.value_counts())

In [14]:
system_results = pd.read_csv("system_results.csv", header=0, sep=",")
qrels = pd.read_csv("qrels.csv", header=0, sep=",")
ir_eval = pd.DataFrame(columns=["system_number","query_number","P@10","R@50","r-precision","AP","nDCG@10","nDCG@20"])
for sys_num in range(1,7):
    qr_dic = dict(system_results[system_results["system_number"] == sys_num].query_number.value_counts())
    ireval = pd.DataFrame(columns=["system_number","query_number","P@10","R@50","r-precision","AP","nDCG@10","nDCG@20"])
    ireval["query_number"] = np.array(range(1,11))
    ireval["system_number"] = np.ones((10,1))*(sys_num)
    sub_results = system_results[system_results["system_number"] == sys_num]

    P_10 = []
    R_50 = []
    r_precision = []
    AP = []
    nDCG_10 = []
    nDCG_20 = []

    index = 0
    for qr_id in range(1,11):
        qrel = list(qrels[qrels["query_id"] == qr_id]["doc_id"])

        result1 = list(sub_results[index: index + 10]["doc_number"])
        P_10.append(len(set(result1) & set(qrel)) / 10)

        result2 = list(sub_results[index: index + 50]["doc_number"])
        R_50.append(len(set(result2) & set(qrel)) / len(qrel))

        result3 = list(sub_results[index: index + len(qrel)]["doc_number"])
        r_precision.append(len(set(result3) & set(qrel)) / len(qrel))

        result4 = sub_results[index: index + qr_dic[qr_id]]
        ap_list = []
        for doc in qrel:
            if doc in list(result4["doc_number"]):
                ap_list.append(int(result4[result4["doc_number"] == doc]["rank_of_doc"]))
        count = 1
        total = 0
        for idx in sorted(ap_list):
            total += count / idx
            count += 1
        AP.append(total/len(qrel))

        result5 = sub_results[index: index + 10]
        DCG10 = 0
        for doc in qrel:
            if doc in list(result5["doc_number"]):
                G = int(qrels.loc[(qrels["query_id"] == qr_id) & (qrels["doc_id"] == doc)]["relevance"])
                if int(result5[result5["doc_number"] == doc]["rank_of_doc"]) == 1:
                    DCG10 += G
                else:
                    DCG10 += G / math.log2(int(result5[result5["doc_number"] == doc]["rank_of_doc"]))

        rel_list = sorted(list(qrels[qrels["query_id"] == qr_id]["relevance"]),reverse=True)
        iDCG_10 = rel_list[0]
        if len(rel_list) <= 10:
            for i in range(1,len(rel_list)):
                iDCG_10 += rel_list[i] / math.log2(i+1)
        else:
            for i in range(1,10):
                iDCG_10 += rel_list[i] / math.log2(i+1)
        nDCG_10.append(DCG10 / iDCG_10)


        result6 = sub_results[index: index + 20]
        DCG20 = 0
        for doc in qrel:
            if doc in list(result6["doc_number"]):
                G = int(qrels.loc[(qrels["query_id"] == qr_id) & (qrels["doc_id"] == doc)]["relevance"])
                if int(result6[result6["doc_number"] == doc]["rank_of_doc"]) == 1:
                    DCG20 += G
                else:
                    DCG20 += G / math.log2(int(result6[result6["doc_number"] == doc]["rank_of_doc"]))

        rel_list = sorted(list(qrels[qrels["query_id"] == qr_id]["relevance"]),reverse=True)
        iDCG_20 = rel_list[0]
        if len(rel_list) <= 20:
            for i in range(1,len(rel_list)):
                iDCG_20 += rel_list[i] / math.log2(i+1)
        else:
            for i in range(1,20):
                iDCG_20 += rel_list[i] / math.log2(i+1)
        nDCG_20.append(DCG20/iDCG_20)

        index += qr_dic[qr_id]


    ireval["P@10"] = np.array(P_10)
    ireval["R@50"] = np.array(R_50)
    ireval["r-precision"] = np.array(r_precision)
    ireval["AP"] = np.array(AP)
    ireval["nDCG@10"] = np.array(nDCG_10)
    ireval["nDCG@20"] = np.array(nDCG_20)
    ireval.index = ireval.index + 1
    ireval.loc["mean"] = ireval.mean()

    ir_eval = pd.concat([ir_eval,ireval], axis=0)


In [15]:
mean_df = ir_eval.loc["mean"]
mean_df.index = range(1,7)
mean_df = mean_df[["P@10","R@50","r-precision","AP","nDCG@10","nDCG@20"]]
mean_df
mean_df[mean_df == mean_df.max()]

,P@10,R@50,r-precision,AP,nDCG@10,nDCG@20
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.867024,NaN,NaN,NaN,NaN
3,0.41,NaN,0.448452,0.451175,0.420348,0.510918
4,NaN,NaN,NaN,NaN,NaN,NaN
5,0.41,NaN,NaN,NaN,NaN,NaN
6,0.41,NaN,0.448452,NaN,NaN,NaN


In [16]:
mean_df[mean_df == mean_df[mean_df != mean_df.max()].max()]

,P@10,R@50,r-precision,AP,nDCG@10,nDCG@20
1,0.39,0.83369,0.401071,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,0.444577,0.400031,0.490532


In [17]:
ir_eval["system_number"] = ir_eval["system_number"].astype(int)
ir_eval["query_number"] = ir_eval["system_number"].astype(int)


In [18]:
from scipy import stats
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 3][:10]["P@10"], ir_eval[ir_eval["system_number"] == 1][:10]["P@10"]))
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 5][:10]["P@10"], ir_eval[ir_eval["system_number"] == 1][:10]["P@10"]))
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 6][:10]["P@10"], ir_eval[ir_eval["system_number"] == 1][:10]["P@10"]))


Ttest_indResult(statistic=0.1422936016353744, pvalue=0.8884287243093304)
Ttest_indResult(statistic=0.1422936016353744, pvalue=0.8884287243093304)
Ttest_indResult(statistic=0.1422936016353744, pvalue=0.8884287243093304)


In [19]:
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 2][:10]["R@50"], ir_eval[ir_eval["system_number"] == 1][:10]["R@50"]))

Ttest_indResult(statistic=0.3880082704800308, pvalue=0.7025603945402291)


In [20]:
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 3][:10]["r-precision"], ir_eval[ir_eval["system_number"] == 1][:10]["r-precision"]))
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 6][:10]["r-precision"], ir_eval[ir_eval["system_number"] == 1][:10]["r-precision"]))

Ttest_indResult(statistic=0.31083959807728023, pvalue=0.7594933627635715)
Ttest_indResult(statistic=0.31083959807728023, pvalue=0.7594933627635715)


In [21]:
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 3][:10]["AP"], ir_eval[ir_eval["system_number"] == 6][:10]["AP"]))

Ttest_indResult(statistic=0.042513564801249584, pvalue=0.9665573458870693)


In [22]:
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 3][:10]["nDCG@10"], ir_eval[ir_eval["system_number"] == 6][:10]["nDCG@10"]))

Ttest_indResult(statistic=0.15065127811699436, pvalue=0.8819262156757446)


In [23]:
print(stats.ttest_ind(ir_eval[ir_eval["system_number"] == 3][:10]["nDCG@20"], ir_eval[ir_eval["system_number"] == 6][:10]["nDCG@20"]))

Ttest_indResult(statistic=0.16786186996140187, pvalue=0.8685635746981927)


In [ ]:
'''
f1 = "ir_eval.csv"
with open(f1, "a+") as file:
    file.write("system_number,query_number,P@10,R@50,r-precision,AP,nDCG@10,nDCG@20" + "\n")
for index, row in ir_eval.iterrows():
    with open(f1, "a+") as file:
        file.write(str(int(row["system_number"])) + "," + str(index) + "," + "{:.3f}".format(row["P@10"])+\
         "," "{:.3f}".format(row["R@50"]) + "," + "{:.3f}".format(row["r-precision"])+\
         "," "{:.3f}".format(row["AP"]) + "," + "{:.3f}".format(row["nDCG@10"]) + "," +\
             "{:.3f}".format(row["nDCG@20"]) + "\n")
'''
    

In [51]:
englishTS = open("englishST.txt")
stopwords = englishTS.readlines()
stopword = []
for word in stopwords:
    stopword.append(word.replace("\n", ""))

In [52]:
data = open("train_and_dev.tsv").read()
lines = data.split("\n")
corpus_Quran = []
corpus_NT = []
corpus_OT = []
Quran_dic = {}
NT_dic = {}
OT_dic = {}
for line in lines:
#    line = line.strip()
    corpus_name, text = line.split("\t")
    
    content_withoutoken = re.sub(r"[^\w]+", " ", text).lower()
    words_list = content_withoutoken.split(" ")
    words = filter(lambda x: x not in stopword and x != "", words_list)       
    terms = [stem(word) for word in words] 
    
    if corpus_name == "Quran":
        corpus_Quran.append(terms)
        for term in set(terms):
            
            if term not in Quran_dic:
                Quran_dic[term] = 1
            else:
                Quran_dic[term] += 1 
                
    elif corpus_name == "NT":
        corpus_NT.append(terms)
        for term in set(terms):
            if term not in NT_dic:
                NT_dic[term] = 1
            else:
                NT_dic[term] += 1 
                
    else:
        corpus_OT.append(terms)
        for term in set(terms):
            if term not in OT_dic:
                OT_dic[term] = 1
            else:
                OT_dic[term] += 1     

In [53]:
from collections import Counter
X,Y, Z=Counter(Quran_dic),Counter(NT_dic),Counter(OT_dic)
total_dic=dict(X+Y+Z)
total_dic



{'world': 305,
 'allah': 1693,
 'prais': 270,
 'lord': 5896,
 'merci': 517,
 'day': 2267,
 'owner': 34,
 'recompens': 129,
 'path': 149,
 'straight': 60,
 'guid': 239,
 'answer': 547,
 'anger': 200,
 'astray': 99,
 'favor': 93,
 'amen': 61,
 'aliflaammeem': 5,
 'guidanc': 70,
 'cautious': 85,
 'book': 354,
 'holi': 465,
 'doubt': 76,
 'evil': 656,
 'hell': 113,
 'prayer': 167,
 'unseen': 43,
 'spend': 46,
 'establish': 143,
 'daili': 45,
 'provid': 74,
 'prophet': 494,
 'everlast': 163,
 'life': 507,
 'mose': 772,
 'jesus': 753,
 'muhammad': 84,
 'firm': 37,
 'prosper': 104,
 'sure': 529,
 'forewarn': 3,
 'disbeliev': 169,
 'punish': 374,
 'heart': 840,
 'ear': 232,
 'dim': 11,
 'great': 839,
 'sight': 279,
 'seal': 64,
 'set': 612,
 'peopl': 1762,
 'believ': 431,
 'sick': 98,
 'pain': 96,
 'lie': 260,
 'increas': 145,
 'land': 1264,
 'reform': 11,
 'repli': 148,
 'corrupt': 87,
 'sens': 12,
 'evildo': 60,
 'knew': 159,
 'fool': 89,
 'meet': 137,
 'follow': 167,
 'devil': 82,
 'mock': 

In [144]:
print(len(corpus_Quran))
print(len(corpus_OT))
print(len(corpus_NT))

5612
16720
5242


In [54]:
def corpus_scores(total_dic,corpus_dic,corpus_Quran,corpus_NT,corpus_OT):
    MI_dic = {}
    Chi_square_dic = {}

    if corpus_dic == Quran_dic:
        X,Y=Counter(OT_dic),Counter(NT_dic)
        other_dic = dict(X+Y)
        total_1 = len(corpus_Quran)
        total_2 = len(corpus_NT) + len(corpus_OT)
    elif corpus_dic == OT_dic:
        X,Y=Counter(Quran_dic),Counter(NT_dic)
        other_dic = dict(X+Y)
        total_1 = len(corpus_OT)
        total_2 = len(corpus_NT) + len(corpus_Quran)
    else: 
        X,Y=Counter(Quran_dic),Counter(OT_dic)
        other_dic = dict(X+Y)
        total_1 = len(corpus_NT)
        total_2 = len(corpus_Quran) + len(corpus_OT)

    N = total_1 + total_2
    for term in total_dic.keys():
        if term in corpus_dic.keys():        
            N11 = corpus_dic[term]
            N01 = total_1 - N11       
            if term in other_dic.keys():
                N10 = other_dic[term]
                N00 = total_2 - N10
                MI = N11/N*math.log2(float(N*N11) / float((N10+N11)*(N01+N11))) \
                 +N01/N*math.log2(float(N*N01) / float((N00+N01)*(N01+N11))) \
                 +N10/N*math.log2(float(N*N10) / float((N10+N11)*(N00+N10))) \
                 +N00/N*math.log2(float(N*N00) / float((N00+N01)*(N00+N10)))
            else:
                N10 = 0
                N00 = total_2
                MI = N11/N*math.log2(float(N*N11)/float((N10+N11)*(N01+N11))) \
                 +N01/N*math.log2(float(N*N01)/float((N00+N01)*(N01+N11))) \
                 +N00/N*math.log2(float(N*N00)/float((N00+N01)*(N00+N10)))
        else:
            N11 = 0
            N01 = total_1 
            N10 = other_dic[term]
            N00 = total_2 - N10
            MI = N01/N*math.log2(float(N*N01) / float((N00+N01)*(N01+N11))) \
                 +N10/N*math.log2(float(N*N10) / float((N10+N11)*(N00+N10))) \
                 +N00/N*math.log2(float(N*N00) / float((N00+N01)*(N00+N10)))
        MI_dic[term] = MI

        Chi_square = ((N11+N10+N01+N00)*math.pow(N11*N00-N10*N01,2)) / ((N11+N01)*(N11+N10)*(N10+N00)*(N01+N00))
        Chi_square_dic[term] = Chi_square
        
    return MI_dic, Chi_square_dic

In [55]:
MI_Quran, Chi_square_Quran = corpus_scores(total_dic,Quran_dic,corpus_Quran,corpus_NT,corpus_OT)
MI_OT, Chi_square_OT = corpus_scores(total_dic,OT_dic,corpus_Quran,corpus_NT,corpus_OT)
MI_NT, Chi_square_NT = corpus_scores(total_dic,NT_dic,corpus_Quran,corpus_NT,corpus_OT)

In [56]:
print(sorted(MI_Quran.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])
print(sorted(Chi_square_Quran.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])

[('allah', 0.15319174926782153), ('thou', 0.037027404268602376), ('thi', 0.028306798113906108), ('ye', 0.026932739870397314), ('thee', 0.026111396777541526), ('god', 0.022404786826180714), ('punish', 0.018751834172204114), ('king', 0.017892721837634826), ('believ', 0.01787681142256401), ('man', 0.017757428677315185)]
[('allah', 7058.784143738311), ('punish', 954.1897716647732), ('believ', 910.8133059378158), ('thou', 834.8478396360407), ('messeng', 819.418048625358), ('unbeliev', 811.8215430936409), ('guid', 686.3372227441748), ('beli', 683.3281897896652), ('vers', 665.4430511260113), ('disbeliev', 665.4430511260113)]


In [57]:
print(sorted(MI_OT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])
print(sorted(Chi_square_OT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])

[('allah', 0.08710753998082718), ('jesus', 0.037588393650712364), ('israel', 0.03235898992840732), ('king', 0.0273995085753877), ('thi', 0.02736681986730472), ('lord', 0.026940775294397652), ('thou', 0.02190644772576018), ('christ', 0.018351984688282773), ('thee', 0.018290912347786646), ('believ', 0.016944716693521752)]
[('allah', 2778.5750550098587), ('jesus', 1192.521553482685), ('lord', 967.9509639503143), ('israel', 956.1034188604813), ('thi', 880.0719696268063), ('king', 828.0399827009664), ('thou', 747.922422689343), ('thee', 611.3603948330435), ('believ', 584.7659158319024), ('christ', 580.8825506237571)]


In [58]:
print(sorted(MI_NT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])
print(sorted(Chi_square_NT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:10])

[('jesus', 0.058731012445188704), ('christ', 0.03288815121293074), ('allah', 0.01934625476268907), ('discipl', 0.015923972541393803), ('lord', 0.015790474873697592), ('faith', 0.012759400565998353), ('ye', 0.012280539725391278), ('israel', 0.011920941418771864), ('paul', 0.011589929154055448), ('peter', 0.01083356460344418)]
[('jesus', 2981.505600401739), ('christ', 1606.4694735818655), ('discipl', 806.0978087426082), ('faith', 674.7384048209634), ('paul', 576.0372334575312), ('ye', 554.0346658679341), ('peter', 530.6518836718143), ('lord', 524.5832111833093), ('thing', 447.2965626664446), ('allah', 423.39436537624914)]


In [59]:
sorted(total_dic.items(), key=lambda item:item[1], reverse=True)

[('lord', 5896),
 ('god', 3242),
 ('thou', 2921),
 ('day', 2267),
 ('thi', 2266),
 ('ye', 2161),
 ('son', 2117),
 ('thee', 2098),
 ('man', 1923),
 ('israel', 1779),
 ('peopl', 1762),
 ('allah', 1693),
 ('king', 1672),
 ('hand', 1463),
 ('hous', 1447),
 ('hath', 1392),
 ('thing', 1350),
 ('men', 1267),
 ('land', 1264),
 ('made', 1231),
 ('father', 1228),
 ('children', 1228),
 ('behold', 1101),
 ('earth', 1065),
 ('word', 1011),
 ('saith', 946),
 ('make', 932),
 ('shalt', 932),
 ('citi', 907),
 ('heart', 840),
 ('great', 839),
 ('good', 838),
 ('pass', 827),
 ('give', 819),
 ('heaven', 782),
 ('call', 774),
 ('mose', 772),
 ('jesus', 753),
 ('put', 738),
 ('brought', 731),
 ('david', 728),
 ('place', 706),
 ('bring', 704),
 ('hast', 690),
 ('servant', 677),
 ('turn', 674),
 ('command', 665),
 ('evil', 656),
 ('nation', 622),
 ('thine', 621),
 ('fear', 618),
 ('year', 616),
 ('set', 612),
 ('time', 607),
 ('priest', 603),
 ('sin', 599),
 ('judah', 594),
 ('offer', 581),
 ('jerusalem', 571

In [40]:
print(len(corpus_Quran))
sorted(Quran_dic.items(), key=lambda item:item[1], reverse=True)

5612


[('allah', 1693),
 ('lord', 777),
 ('day', 433),
 ('believ', 338),
 ('earth', 329),
 ('peopl', 320),
 ('punish', 315),
 ('sure', 312),
 ('messeng', 296),
 ('nation', 294),
 ('merci', 271),
 ('good', 246),
 ('heaven', 243),
 ('turn', 230),
 ('worship', 229),
 ('fear', 225),
 ('unbeliev', 216),
 ('guid', 211),
 ('book', 203),
 ('evil', 196),
 ('made', 196),
 ('truth', 191),
 ('thing', 189),
 ('forgiv', 184),
 ('sign', 183),
 ('clear', 180),
 ('beli', 175),
 ('give', 174),
 ('life', 173),
 ('disbeliev', 169),
 ('vers', 169),
 ('order', 165),
 ('prophet', 161),
 ('creat', 161),
 ('call', 150),
 ('knowledg', 150),
 ('repli', 148),
 ('make', 148),
 ('bring', 143),
 ('mose', 140),
 ('fire', 138),
 ('heart', 135),
 ('return', 135),
 ('whosoev', 118),
 ('god', 118),
 ('world', 113),
 ('rememb', 113),
 ('recompens', 108),
 ('promis', 108),
 ('soul', 106),
 ('everlast', 104),
 ('live', 104),
 ('harmdoer', 104),
 ('sin', 103),
 ('path', 101),
 ('hand', 101),
 ('land', 100),
 ('belong', 100),
 ('ex

In [41]:
print(len(corpus_OT))
sorted(OT_dic.items(), key=lambda item:item[1], reverse=True)

16720


[('lord', 4610),
 ('thou', 2454),
 ('god', 2241),
 ('thi', 2035),
 ('thee', 1804),
 ('son', 1753),
 ('israel', 1695),
 ('king', 1571),
 ('day', 1488),
 ('ye', 1338),
 ('man', 1273),
 ('peopl', 1270),
 ('hous', 1251),
 ('hand', 1168),
 ('land', 1123),
 ('children', 1039),
 ('hath', 1036),
 ('men', 906),
 ('behold', 899),
 ('shalt', 864),
 ('father', 835),
 ('made', 834),
 ('citi', 756),
 ('saith', 716),
 ('word', 697),
 ('david', 670),
 ('pass', 668),
 ('make', 664),
 ('thing', 607),
 ('earth', 600),
 ('hast', 596),
 ('judah', 593),
 ('heart', 583),
 ('thine', 571),
 ('mose', 570),
 ('brought', 562),
 ('servant', 562),
 ('place', 554),
 ('great', 554),
 ('put', 545),
 ('year', 522),
 ('offer', 518),
 ('thereof', 495),
 ('priest', 487),
 ('give', 485),
 ('bring', 481),
 ('command', 477),
 ('jerusalem', 465),
 ('set', 462),
 ('ofth', 438),
 ('mine', 417),
 ('call', 409),
 ('good', 407),
 ('water', 404),
 ('time', 403),
 ('eye', 399),
 ('egypt', 394),
 ('turn', 391),
 ('spake', 378),
 ('ho

In [42]:
print(len(corpus_NT))
sorted(NT_dic.items(), key=lambda item:item[1], reverse=True)

5242


[('god', 883),
 ('ye', 823),
 ('jesus', 723),
 ('man', 620),
 ('thing', 554),
 ('lord', 509),
 ('thou', 467),
 ('christ', 372),
 ('hath', 356),
 ('day', 346),
 ('men', 309),
 ('father', 300),
 ('thee', 294),
 ('son', 277),
 ('word', 233),
 ('thi', 231),
 ('saith', 230),
 ('call', 215),
 ('heaven', 213),
 ('faith', 213),
 ('spirit', 212),
 ('heard', 208),
 ('made', 201),
 ('behold', 197),
 ('discipl', 195),
 ('great', 194),
 ('hand', 194),
 ('answer', 194),
 ('receiv', 189),
 ('2', 185),
 ('good', 185),
 ('brethren', 183),
 ('peopl', 172),
 ('world', 162),
 ('law', 162),
 ('give', 160),
 ('hous', 155),
 ('love', 154),
 ('4', 152),
 ('3', 151),
 ('jew', 150),
 ('time', 147),
 ('1', 146),
 ('sin', 145),
 ('work', 142),
 ('holi', 138),
 ('angel', 137),
 ('life', 137),
 ('earth', 136),
 ('paul', 136),
 ('dead', 135),
 ('6', 132),
 ('citi', 131),
 ('cast', 129),
 ('speak', 128),
 ('pass', 127),
 ('glori', 125),
 ('5', 124),
 ('peter', 124),
 ('put', 122),
 ('heart', 122),
 ('prophet', 121),


In [60]:
total_corpus = corpus_Quran + corpus_NT + corpus_OT


dictionary = Dictionary(total_corpus)
dictionary.filter_extremes(no_below=50, no_above=0.1)
corpus = [dictionary.doc2bow(text) for text in total_corpus]
lda = LdaModel(corpus, num_topics=20, id2word=dictionary, random_state=1) 
topics = lda.get_document_topics(corpus)

dictionary1 = Dictionary(corpus_Quran)
dictionary1.filter_extremes(no_below=50, no_above=0.1)
corpus1 = [dictionary1.doc2bow(text) for text in corpus_Quran]
topics_Quran = lda.get_document_topics(corpus1)
topic_dic_Quran = {}
for doc in topics_Quran:
    for topic in doc:
        if topic[0] not in topic_dic_Quran.keys():
            topic_dic_Quran[topic[0]] = topic[1]
        else:
            topic_dic_Quran[topic[0]] += topic[1]
            
            
dictionary2 = Dictionary(corpus_OT)
dictionary2.filter_extremes(no_below=50,no_above=0.1)
corpus2 = [dictionary2.doc2bow(text) for text in corpus_OT]
topics_OT = lda.get_document_topics(corpus2)
topic_dic_OT = {}
for doc in topics_OT:
    for topic in doc:
        if topic[0] not in topic_dic_OT.keys():
            topic_dic_OT[topic[0]] = topic[1]
        else:
            topic_dic_OT[topic[0]] += topic[1]
            
                        
dictionary3 = Dictionary(corpus_NT)
dictionary3.filter_extremes(no_below=50,no_above=0.1)
corpus3 = [dictionary3.doc2bow(text) for text in corpus_NT]
topics_NT = lda.get_document_topics(corpus3)
topic_dic_NT = {}
for doc in topics_NT:
    for topic in doc:
        if topic[0] not in topic_dic_NT.keys():
            topic_dic_NT[topic[0]] = topic[1]
        else:
            topic_dic_NT[topic[0]] += topic[1]

for k,v in topic_dic_Quran.items():
    topic_dic_Quran[k] = v / len(corpus_Quran)
for k,v in topic_dic_OT.items():
    topic_dic_OT[k] = v / len(corpus_OT)
for k,v in topic_dic_NT.items():
    topic_dic_NT[k] = v / len(corpus_NT)
    
topic_ranked_Quran = sorted(topic_dic_Quran.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:5]
for topic in topic_ranked_Quran:
    print("topic_id: " + str(topic[0]) + ", score: " + str(topic[1]))
    print(lda.print_topic(topic[0]))
print("\n")    
topic_ranked_OT = sorted(topic_dic_OT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:5]
for topic in topic_ranked_OT:
    print("topic_id: " + str(topic[0]) + ", score: " + str(topic[1]))
    print(lda.print_topic(topic[0]))
print("\n")       
topic_ranked_NT = sorted(topic_dic_NT.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:5]
for topic in topic_ranked_NT:
    print("topic_id: " + str(topic[0]) + ", score: " + str(topic[1]))
    print(lda.print_topic(topic[0]))
  

topic_id: 16, score: 0.0634973679622071
0.096*"year" + 0.067*"voic" + 0.063*"cri" + 0.061*"thousand" + 0.054*"hundr" + 0.049*"month" + 0.048*"walk" + 0.038*"stone" + 0.037*"young" + 0.034*"men"
topic_id: 12, score: 0.0630742873112304
0.079*"wick" + 0.074*"command" + 0.065*"lie" + 0.058*"strength" + 0.055*"blood" + 0.051*"sacrific" + 0.049*"reign" + 0.041*"wrath" + 0.023*"comfort" + 0.022*"bed"
topic_id: 4, score: 0.05909284182471712
0.298*"israel" + 0.160*"children" + 0.059*"mouth" + 0.038*"love" + 0.038*"bless" + 0.037*"lift" + 0.030*"pray" + 0.029*"merci" + 0.025*"feet" + 0.022*"increas"
topic_id: 6, score: 0.05688467861836244
0.155*"men" + 0.105*"host" + 0.063*"work" + 0.051*"speak" + 0.048*"round" + 0.041*"father" + 0.038*"peac" + 0.033*"made" + 0.030*"hous" + 0.022*"good"
topic_id: 15, score: 0.055924119901759065
0.309*"day" + 0.058*"gather" + 0.054*"sea" + 0.037*"save" + 0.037*"life" + 0.032*"night" + 0.032*"land" + 0.030*"enter" + 0.027*"brought" + 0.023*"peopl"


topic_id: 13, 

In [96]:
#print(OT_dic["thou"])
frequence("")

329
600
136


In [84]:
def frequence(term):
    
    print(Quran_dic[term])
    print(OT_dic[term])
    print(NT_dic[term])

In [120]:
X, Y, Z = Counter(Chi_square_Quran), Counter(Chi_square_NT), Counter(Chi_square_OT)
total_Chi_square = dict(X + Y + Z)
Chi_square_list = []
for i in sorted(total_Chi_square.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:5000]:
    Chi_square_list.append(i[0])

In [130]:
X, Y, Z = Counter(MI_Quran), Counter(MI_NT), Counter(MI_OT)
total_MI = dict(X + Y + Z)
MI_list = []
for i in sorted(total_MI.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)[:5000]:
    MI_list.append(i[0])

In [124]:
len(sorted(total_Chi_square.items(), key = lambda kv:(kv[1], kv[0]), reverse=True))

28619

In [2]:
def preprocess_data(data):
    chars_to_remove = re.compile(f'[{string.punctuation}]')
    
    documents = []
    categories = []
    vocab = set([])
    
    lines = data.split("\n")
    
    for line in lines:
        line = line.strip()
        if line:
            category, text = line.split("\t")
            words = chars_to_remove.sub("", text).lower().split()
            for word in words:
                vocab.add(word)
            documents.append(words)
            categories.append(category)
            
    return documents, categories, vocab


In [7]:
word2id = {}
for word_id, word in enumerate(vocab):
    word2id[word] = word_id
    
cat2id = {}
for cat_id, cat in enumerate(set(categories)):
    cat2id[cat] = cat_id

In [8]:
def convert_to_bow_matrix(preprocessed_data, word2id):
    matrix_size = (len(preprocessed_data), len(word2id)+1)
    oov_index = len(word2id)
    X = scipy.sparse.dok_matrix(matrix_size)
    
    for doc_id, doc in enumerate(preprocessed_data):
        for word in doc:
            X[doc_id, word2id.get(word, oov_index)] += 1
    
    return X


In [99]:
def accuracy_check(y_true, y_pred, category, cat2id):  
    #true positive
    TP = np.sum(np.logical_and(np.equal(y_true,cat2id[category]),np.equal(y_pred,cat2id[category])))
    #false positive
    FP = np.sum(np.logical_and(np.not_equal(y_true,cat2id[category]),np.equal(y_pred,cat2id[category])))
    #true negative
    TN = np.sum(np.logical_and(np.not_equal(y_true,cat2id[category]),np.not_equal(y_pred,cat2id[category])))
    #false negative    
    FN = np.sum(np.logical_and(np.equal(y_true,cat2id[category]),np.not_equal(y_pred,cat2id[category])))
    
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    f1 = 2 * p * r / (p + r)
    return round(p,3), round(r,3), round(f1,3)


In [139]:
def print_accuracy(y_true, y_pred, cat2id):
    p_Quran, r_Quran, f_Quran = accuracy_check(y_true, y_pred, "Quran", cat2id)
    p_NT, r_NT, f_NT = accuracy_check(y_true, y_pred,"NT", cat2id)
    p_OT, r_OT, f_OT = accuracy_check(y_true, y_pred,"OT", cat2id)
    p_macro = round((p_Quran + p_NT + p_OT) / 3, 3)
    r_macro = round((r_Quran + r_NT + r_OT) / 3, 3)
    f_macro = round((f_Quran + f_NT + f_OT) / 3, 3)
    accuracy_list = [str(p_Quran), str(r_Quran), str(f_Quran), str(p_NT), str(r_NT), str(f_NT), str(p_OT), str(r_OT), str(f_OT), str(p_macro), str(r_macro), str(f_macro)]
    return accuracy_list

In [140]:
l = print_accuracy(y_test_predictions, y_test, cat2id)

In [141]:
','.join(l)

'0.809,0.825,0.817,0.662,0.666,0.664,0.872,0.864,0.868,0.781,0.785,0.783'

In [102]:
def Train_Dev_split(preprocessed_data, categories):
    preprocessed_training_data = []
    training_categories = []
    preprocessed_dev_data = []
    dev_categories = []
    random.seed(1)
    dev_index = [random.randint(0, len(preprocessed_data)) for _ in range(len(preprocessed_data) // 10)]
    for i in dev_index:
        preprocessed_dev_data.append(preprocessed_data[i])
        dev_categories.append(categories[i])

    train_index = [i for i in range(len(preprocessed_data)) if i not in dev_index]
    for i in train_index:
        preprocessed_training_data.append(preprocessed_data[i])
        training_categories.append(categories[i])

    return preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index 

In [113]:
import random
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = preprocess_data(training_data)


preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index = Train_Dev_split(preprocessed_data, categories)
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=1000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = preprocess_data(testing_data)
X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

0.998 1.0 0.999
1.0 0.997 0.998
1.0 1.0 1.0
0.999 0.999 0.999


0.879 0.912 0.895
0.804 0.753 0.778
0.92 0.927 0.924
0.868 0.864 0.866


0.893 0.949 0.92
0.834 0.801 0.817
0.938 0.93 0.934
0.888 0.893 0.89


In [110]:
import random
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = preprocess_data(training_data)


preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index = Train_Dev_split(preprocessed_data, categories)
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=100,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = preprocess_data(testing_data)
X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

0.989 0.992 0.99
0.99 0.956 0.973
0.987 0.997 0.992
0.989 0.982 0.985


0.908 0.926 0.917
0.857 0.74 0.794
0.918 0.952 0.935
0.894 0.873 0.882


0.902 0.949 0.925
0.864 0.796 0.828
0.934 0.941 0.937
0.9 0.895 0.897


In [111]:
import random
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = preprocess_data(training_data)


preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index = Train_Dev_split(preprocessed_data, categories)
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=10000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = preprocess_data(testing_data)
X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

1.0 1.0 1.0
1.0 1.0 1.0
1.0 1.0 1.0
1.0 1.0 1.0


0.87 0.902 0.886
0.777 0.749 0.763
0.917 0.917 0.917
0.855 0.856 0.855


0.875 0.931 0.902
0.817 0.782 0.799
0.928 0.92 0.924
0.873 0.878 0.875


In [112]:
def check_incorrect(predictions, true_values):
    incorrect = []
    correct = []
    for i in range(len(predictions)):
        if predictions[i] != true_values[i]:
            incorrect.append(i)
    return incorrect

In [118]:
incorrect = check_incorrect(y_dev_predictions, y_dev)
for i in incorrect[:3]:
    print(preprocessed_data[dev_index[i]])
    print("predicted category: " + list(cat2id.keys())[list(cat2id.values()).index(y_dev_predictions[i])])
    print("true category: " + list(cat2id.keys())[list(cat2id.values()).index(y_dev[i])])

['in', 'a', 'secure', 'abode', 'in', 'the', 'presence', 'of', 'the', 'powerful', 'king']
predicted category: OT
true category: Quran
['and', 'they', 'stirred', 'up', 'the', 'people', 'and', 'the', 'elders', 'and', 'the', 'scribesand', 'came', 'upon', 'him', 'and', 'caught', 'him', 'and', 'brought', 'him', 'to', 'the', 'council613', 'and', 'set', 'up', 'false', 'witnesses', 'which', 'said', 'this', 'man', 'ceaseth', 'not', 'tospeak', 'blasphemous', 'words', 'against', 'this', 'holy', 'place', 'and', 'the', 'law', '614', 'forwe', 'have', 'heard', 'him', 'say', 'that', 'this', 'jesus', 'of', 'nazareth', 'shall', 'destroy', 'thisplace', 'and', 'shall', 'change', 'the', 'customs', 'which', 'moses', 'delivered', 'us']
predicted category: OT
true category: NT
['and', 'among', 'his', 'signs', 'are', 'the', 'ships', 'that', 'run', 'on', 'the', 'sea', 'like', 'mountains', 'and']
predicted category: OT
true category: Quran


In [106]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

model = OneVsRestClassifier(LogisticRegression())
clf = model.fit(X_train, y_train)
clf.fit(X_train, y_train)

y_train_predictions = clf.predict(X_train)
y_dev_predictions = clf.predict(X_dev)
y_test_predictions = clf.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

0.978 0.961 0.969
0.952 0.85 0.898
0.948 0.985 0.966
0.959 0.932 0.944


0.919 0.923 0.921
0.859 0.719 0.783
0.912 0.958 0.935
0.897 0.867 0.88


0.945 0.907 0.926
0.893 0.785 0.835
0.919 0.966 0.942
0.919 0.886 0.901


In [131]:
def improved_preprocess_data(data):
#    total_words = list(set(corpus_NT) + set(corpus_OT) + set(corpus_Quran))
    chars_to_remove = re.compile(f'[{string.punctuation}]')
    
    documents = []
    categories = []
    vocab = set([])
    
    lines = data.split("\n")
    
    for line in lines:
        line = line.strip()
        if line:
            category, text = line.split("\t")
            words_list = chars_to_remove.sub("", text).lower().split()
#            words = list(filter(lambda x: x not in stopword, words_list))  
#            words = [stem(word) for word in words_list] 
#            words = [word for word in words if word in total]
            words = [word for word in words_list if word in MI_list]
            
            for word in words:
                vocab.add(word)
            documents.append(words)
            categories.append(category)
            
    return documents, categories, vocab

#training_data = open("train_and_dev.tsv").read()
#preprocessed_data, categories, vocab = improved_preprocess_data(training_data)

In [65]:
import random
preprocessed_training_data = []
training_categories = []
preprocessed_dev_data = []
dev_categories = []
random.seed(1)
dev_index = [random.randint(0,len(preprocessed_data)) for _ in range(len(preprocessed_data)//10)]
for i in dev_index:
    preprocessed_dev_data.append(preprocessed_data[i])
    dev_categories.append(categories[i])
    
train_index = [i for i in range(len(preprocessed_data)) if i not in dev_index]
for i in train_index:
    preprocessed_training_data.append(preprocessed_data[i])
    training_categories.append(categories[i])
    

X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)

word2id = {}
for word_id, word in enumerate(vocab):
    word2id[word] = word_id
    
cat2id = {}
for cat_id, cat in enumerate(set(categories)):
    cat2id[cat] = cat_id
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=1000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)

testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = improved_preprocess_data(testing_data)

X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)


print(classification_report(y_dev, y_dev_predictions, target_names=["Quran","OT","NT"]))
print(classification_report(y_test, y_test_predictions, target_names=["Quran","OT","NT"]))

              precision    recall  f1-score   support

       Quran       0.89      0.94      0.92      1636
          OT       0.90      0.89      0.89       601
          NT       0.84      0.69      0.76       520

    accuracy                           0.88      2757
   macro avg       0.88      0.84      0.86      2757
weighted avg       0.88      0.88      0.88      2757

              precision    recall  f1-score   support

       Quran       0.89      0.96      0.93      1858
          OT       0.90      0.89      0.90       624
          NT       0.87      0.68      0.76       582

    accuracy                           0.89      3064
   macro avg       0.89      0.84      0.86      3064
weighted avg       0.89      0.89      0.89      3064



In [ ]:
    for query_num in range(len(query_terms)):
        doc_dic[query_num] = {}
        for word in query_terms[query_num][1:]:
            doc_num = query_num + 1
            list_docs = query_request(word)
            #document frequency
            df = len(term_dic[word])
            for doc in list_docs:
                if doc not in doc_dic[query_num]:
                    doc_dic[query_num][doc] = 0
                #term frequency
                tf = len(term_dic[word][str(doc)])
                w = (1 + math.log10(tf)) * math.log10(total_num / df)
                doc_dic[query_num][doc] += word

In [88]:
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = preprocess_data(training_data)

preprocessed_training_data = []
training_categories = []
preprocessed_dev_data = []
dev_categories = []
random.seed(0)
dev_index = [random.randint(0,len(preprocessed_data)) for _ in range(len(preprocessed_data)//10)]
for i in dev_index:
    preprocessed_dev_data.append(preprocessed_data[i])
    dev_categories.append(categories[i])
    
train_index = [i for i in range(len(preprocessed_data)) if i not in dev_index]
for i in train_index:
    preprocessed_training_data.append(preprocessed_data[i])
    training_categories.append(categories[i])

    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=10000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


p_Quran_dev, r_Quran_dev, f_Quran_dev = accuracy_check(y_dev,y_dev_predictions,"Quran", cat2id)
p_NT_dev, r_NT_dev, f_NT_dev = accuracy_check(y_dev,y_dev_predictions,"NT", cat2id)
p_OT_dev, r_OT_dev, f_OT_dev = accuracy_check(y_dev,y_dev_predictions,"OT", cat2id)
p_macro_dev = (p_Quran_dev + p_NT_dev + p_OT_dev) / 3
r_macro_dev = (r_Quran_dev + r_NT_dev + r_OT_dev) / 3
f_macro_dev = (f_Quran_dev + f_NT_dev + f_OT_dev) / 3

print(p_Quran_dev, r_Quran_dev, f_Quran_dev)
print(p_NT_dev, r_NT_dev, f_NT_dev)
print(p_OT_dev, r_OT_dev, f_OT_dev)
print(p_macro_dev, r_macro_dev, f_macro_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = preprocess_data(testing_data)


p_Quran_test, r_Quran_test, f_Quran_test = accuracy_check(y_test,y_test_predictions,"Quran", cat2id)
p_NT_test, r_NT_test, f_NT_test = accuracy_check(y_test,y_test_predictions,"NT", cat2id)
p_OT_test, r_OT_test, f_OT_test = accuracy_check(y_test,y_test_predictions,"OT", cat2id)
p_macro_test = (p_Quran_test + p_NT_test + p_OT_test) / 3
r_macro_test = (r_Quran_test + r_NT_test + r_OT_test) / 3
f_macro_test = (f_Quran_test + f_NT_test + f_OT_test) / 3

print("\n")
print(p_Quran_test, r_Quran_test, f_Quran_test)
print(p_NT_test, r_NT_test, f_NT_test)
print(p_OT_test, r_OT_test, f_OT_test)
print(p_macro_test, r_macro_test, f_macro_test)

0.9003267973856209 0.9168053244592346 0.9084913437757626
0.8083832335329342 0.7788461538461539 0.7933398628795297
0.9209245742092458 0.9254278728606357 0.923170731707317
0.8765448683759337 0.8736931170553414 0.8750006461208697


0.9485903814262023 0.9166666666666666 0.9323553382233088
0.8878143133462283 0.788659793814433 0.835304822565969
0.9212962962962963 0.9639397201291712 0.9421357180431352
0.9192336636895756 0.8897553935367569 0.9032652929441377


In [129]:
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = improved_preprocess_data(training_data)


preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index = Train_Dev_split(preprocessed_data, categories)
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=1000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = improved_preprocess_data(testing_data)
X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

0.926 0.988 0.956
0.989 0.956 0.972
0.993 0.981 0.987
0.969 0.975 0.972


0.764 0.832 0.796
0.619 0.605 0.612
0.858 0.839 0.848
0.747 0.759 0.752


0.825 0.811 0.818
0.661 0.658 0.66
0.861 0.868 0.864
0.782 0.779 0.781


In [132]:
training_data = open("train_and_dev.tsv").read()
preprocessed_data, categories, vocab = improved_preprocess_data(training_data)


preprocessed_training_data, training_categories, preprocessed_dev_data, dev_categories, dev_index = Train_Dev_split(preprocessed_data, categories)
    
X_train = convert_to_bow_matrix(preprocessed_training_data, word2id)
y_train = [cat2id[cat] for cat in training_categories]

model = sklearn.svm.SVC(C=1000,random_state=0)
model.fit(X_train,y_train)

X_dev = convert_to_bow_matrix(preprocessed_dev_data, word2id)
y_dev = [cat2id[cat] for cat in dev_categories]
y_train_predictions = model.predict(X_train)
y_dev_predictions = model.predict(X_dev)


testing_data = open("test.tsv").read()
preprocessed_test_data, test_categories, test_vocab = improved_preprocess_data(testing_data)
X_test = convert_to_bow_matrix(preprocessed_test_data, word2id)
y_test = [cat2id[cat] for cat in test_categories]
y_test_predictions = model.predict(X_test)

print_accuracy(y_train,y_train_predictions, cat2id)
print("\n")
print_accuracy(y_dev,y_dev_predictions, cat2id)
print("\n")
print_accuracy(y_test,y_test_predictions, cat2id)

0.926 0.989 0.956
0.989 0.954 0.971
0.993 0.981 0.987
0.969 0.975 0.971


0.753 0.825 0.787
0.611 0.609 0.61
0.855 0.83 0.842
0.74 0.755 0.746


0.825 0.809 0.817
0.666 0.662 0.664
0.864 0.872 0.868
0.785 0.781 0.783
